# Install Necessary Packages

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=0ee8268aa485b5d46ffb786e313128129a9f8c65b273a0d235f44ee2622e2109
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# Necessary Packages

In [ ]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import StringType,IntegerType,StructField,StructType

# Mount the drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setup

In [ ]:
nomappli = "essai1"
config = SparkConf().setAppName(nomappli)
sc = SparkContext(conf=config)

In [ ]:
sqlContext = SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Data Loading

In [ ]:
file = sc.textFile("./drive/MyDrive/bdt/ngram.csv")
table = file \
  .map(lambda row: row.split("\t")) \
  .map(lambda row: (row[0], int(row[1]),int(row[2]),int(row[3]),int(row[4])))

# Schema definition

In [ ]:
ngram = StructField("ngram", StringType())
year = StructField("year", IntegerType())
count = StructField("count", IntegerType())
pages = StructField("pages", IntegerType())
books = StructField("books", IntegerType())

In [ ]:
schema = StructType([ngram,year,count,pages,books])

# Create a dataframe

In [ ]:
ngram = sqlContext.createDataFrame(data=table, schema=schema)

In [ ]:
ngram.columns

['ngram', 'year', 'count', 'pages', 'books']

In [ ]:
ngram.registerTempTable("ngram")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


# Queries

## Return all the bigrams where the the count is greater than 5

- SQL

In [ ]:
sqlContext.sql("""
  SELECT DISTINCT ngram from ngram as n
  WHERE n.count > 5
""").show()

+--------+
|   ngram|
+--------+
|! $17.95|
|    ! 09|
+--------+



- PySQL

In [ ]:
ngram.filter("count > 5") \
  .select(ngram.ngram) \
  .distinct() \
  .show()

+--------+
|   ngram|
+--------+
|! $17.95|
|    ! 09|
+--------+



## Return the number of bigrams for each year

- SQL

In [ ]:
sqlContext.sql("""
  SELECT n.year,count(n.ngram) as count from ngram as n
  GROUP BY n.year
""").show()

+----+-----+
|year|count|
+----+-----+
|1829|    1|
|1990|    2|
|1884|    1|
|2003|    2|
|1823|    1|
|2007|    2|
|1869|    1|
|1866|    1|
|1877|    1|
|2006|    2|
|1824|    1|
|1848|    1|
|1878|    1|
|1834|    1|
|1997|    2|
|1847|    1|
|1806|    1|
|1856|    1|
|1841|    1|
|1861|    1|
+----+-----+
only showing top 20 rows



- PySQL

In [ ]:
ngram \
  .groupBy("year") \
  .count() \
  .show()

+----+-----+
|year|count|
+----+-----+
|1829|    1|
|1990|    2|
|1884|    1|
|2003|    2|
|1823|    1|
|2007|    2|
|1869|    1|
|1866|    1|
|1877|    1|
|2006|    2|
|1824|    1|
|1848|    1|
|1878|    1|
|1834|    1|
|1997|    2|
|1847|    1|
|1806|    1|
|1856|    1|
|1841|    1|
|1861|    1|
+----+-----+
only showing top 20 rows



## Return the bigrams with highest count each year

- SQL

In [ ]:
sqlContext.sql("""
  SELECT ngram,year,max(count) as count FROM ngram as n
  GROUP BY ngram,year
  ORDER BY year
""").show()

+-----+----+-----+
|ngram|year|count|
+-----+----+-----+
| ! 09|1780|    1|
| ! 09|1803|    1|
| ! 09|1806|    1|
| ! 09|1823|    1|
| ! 09|1824|    1|
| ! 09|1825|    1|
| ! 09|1829|    3|
| ! 09|1830|    2|
| ! 09|1831|    1|
| ! 09|1833|    1|
| ! 09|1834|    4|
| ! 09|1836|    1|
| ! 09|1839|    1|
| ! 09|1840|    1|
| ! 09|1841|    2|
| ! 09|1845|    1|
| ! 09|1847|    2|
| ! 09|1848|    1|
| ! 09|1856|    1|
| ! 09|1857|    2|
+-----+----+-----+
only showing top 20 rows



- PySQL

In [ ]:
ngram \
  .groupBy(["ngram",'year']) \
  .max("count") \
  .sort(["year"]) \
  .show()

+-----+----+----------+
|ngram|year|max(count)|
+-----+----+----------+
| ! 09|1780|         1|
| ! 09|1803|         1|
| ! 09|1806|         1|
| ! 09|1823|         1|
| ! 09|1824|         1|
| ! 09|1825|         1|
| ! 09|1829|         3|
| ! 09|1830|         2|
| ! 09|1831|         1|
| ! 09|1833|         1|
| ! 09|1834|         4|
| ! 09|1836|         1|
| ! 09|1839|         1|
| ! 09|1840|         1|
| ! 09|1841|         2|
| ! 09|1845|         1|
| ! 09|1847|         2|
| ! 09|1848|         1|
| ! 09|1856|         1|
| ! 09|1857|         2|
+-----+----+----------+
only showing top 20 rows



## Return the bigrams that appeared in 20 different year.

- SQL

In [ ]:
sqlContext.sql("""
  SELECT ngram,COUNT(DISTINCT year) as years_count
  FROM ngram as n
  GROUP BY ngram
  HAVING years_count >= 20
""").show()

+--------+-----------+
|   ngram|years_count|
+--------+-----------+
|    ! 09|        100|
|! $17.95|         20|
+--------+-----------+



- PySpark

In [ ]:
ngram \
  .groupBy(ngram.ngram) \
  .count() \
  .filter("count >= 20") \
  .show()

+--------+-----+
|   ngram|count|
+--------+-----+
|    ! 09|  100|
|! $17.95|   20|
+--------+-----+



## Return the bigrams where `!` is the first character and `9` is the second (separated by white space).

- SQL

In [ ]:
sqlContext.sql("""
  SELECT DISTINCT ngram FROM ngram as n
  WHERE n.ngram LIKE '%!% %9%'
""").show()

+--------+
|   ngram|
+--------+
|    ! 09|
|! $17.95|
+--------+



- PySpark

In [ ]:
ngram \
  .filter(ngram.ngram.like('%!% %9%')) \
  .select(ngram.ngram) \
  .distinct() \
  .show()

+--------+
|   ngram|
+--------+
|    ! 09|
|! $17.95|
+--------+



## Return the bigrams that appears in all the years

- SQL

In [ ]:
sqlContext.sql("""
  SELECT ngram,count(DISTINCT year) as years_count FROM ngram as n
  GROUP BY ngram
  HAVING  years_count = (SELECT count(DISTINCT year) as years_count FROM ngram)
""").show()

+-----+-----------+
|ngram|years_count|
+-----+-----------+
| ! 09|        100|
+-----+-----------+



- PySpark

In [ ]:
ngram.select(ngram.ngram,ngram.year) \
  .groupBy(ngram.ngram) \
  .count() \
  .filter(col('count') == ngram.select(ngram.year).distinct().count()) \
  .show()

+-----+-----+
|ngram|count|
+-----+-----+
| ! 09|  100|
+-----+-----+



## Return the total number of pages & books for each bigram by year in alphabetical order

- SQL

In [ ]:
sqlContext.sql("""
  SELECT ngram,SUM(pages),SUM(books),year
  FROM ngram as n
  GROUP BY year,ngram
  ORDER BY ngram DESC
""").show()

+-----+----------+----------+----+
|ngram|sum(pages)|sum(books)|year|
+-----+----------+----------+----+
| ! 09|         1|         1|1860|
| ! 09|         1|         1|1867|
| ! 09|         2|         2|1877|
| ! 09|         3|         3|1829|
| ! 09|         1|         1|1845|
| ! 09|         1|         1|1887|
| ! 09|         2|         2|1876|
| ! 09|         1|         1|1823|
| ! 09|         2|         2|1830|
| ! 09|         1|         1|1825|
| ! 09|         2|         2|1885|
| ! 09|         5|         5|1884|
| ! 09|         2|         2|1879|
| ! 09|         1|         1|1848|
| ! 09|         1|         1|1858|
| ! 09|         2|         2|1878|
| ! 09|         1|         1|1861|
| ! 09|         1|         1|1866|
| ! 09|         2|         2|1880|
| ! 09|         1|         1|1836|
+-----+----------+----------+----+
only showing top 20 rows



- PySQL

In [ ]:
ngram \
  .select('ngram','year','books','pages') \
  .groupBy('year','ngram') \
  .agg(sum('books').alias('total_books'),sum('pages').alias('total_pages')) \
  .sort(desc('ngram')) \
  .show()

+----+-----+-----------+-----------+
|year|ngram|total_books|total_pages|
+----+-----+-----------+-----------+
|1860| ! 09|          1|          1|
|1867| ! 09|          1|          1|
|1877| ! 09|          2|          2|
|1829| ! 09|          3|          3|
|1845| ! 09|          1|          1|
|1887| ! 09|          1|          1|
|1876| ! 09|          2|          2|
|1823| ! 09|          1|          1|
|1830| ! 09|          2|          2|
|1825| ! 09|          1|          1|
|1885| ! 09|          2|          2|
|1884| ! 09|          5|          5|
|1879| ! 09|          2|          2|
|1848| ! 09|          1|          1|
|1858| ! 09|          1|          1|
|1878| ! 09|          2|          2|
|1861| ! 09|          1|          1|
|1866| ! 09|          1|          1|
|1880| ! 09|          2|          2|
|1836| ! 09|          1|          1|
+----+-----+-----------+-----------+
only showing top 20 rows



## Return the number of bigrams by year ordered by year

- SQL

In [ ]:
sqlContext.sql("""
  SELECT year,count(DISTINCT ngram) as count
  FROM ngram as n
  GROUP BY year
  ORDER BY year DESC
""").show()

+----+-----+
|year|count|
+----+-----+
|2008|    2|
|2007|    2|
|2006|    2|
|2005|    2|
|2004|    2|
|2003|    2|
|2002|    2|
|2001|    2|
|2000|    2|
|1999|    2|
|1998|    2|
|1997|    2|
|1996|    2|
|1995|    2|
|1993|    2|
|1992|    2|
|1991|    2|
|1990|    2|
|1987|    2|
|1985|    2|
+----+-----+
only showing top 20 rows



- PySpark

In [ ]:
ngram \
  .select('year','ngram') \
  .groupBy('year') \
  .count() \
  .sort(desc('year')) \
  .show()

+----+-----+
|year|count|
+----+-----+
|2008|    2|
|2007|    2|
|2006|    2|
|2005|    2|
|2004|    2|
|2003|    2|
|2002|    2|
|2001|    2|
|2000|    2|
|1999|    2|
|1998|    2|
|1997|    2|
|1996|    2|
|1995|    2|
|1993|    2|
|1992|    2|
|1991|    2|
|1990|    2|
|1987|    2|
|1985|    2|
+----+-----+
only showing top 20 rows

